<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/Llama_3_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune and Convert Llama3-8B to a GGUF Model File for CPU and GPU Inference 🚀

Hi everyone!

Over the past few months, open-source AI models have taken the community by storm. However, there’s been a lack of comprehensive guides showing how to fine-tune a model and convert it into a format ready for deployment on both CPU and GPU setups. So, we’ve created one for you! 🎉

## In this guide, we’ll cover the following steps:

1. **Fine-tune Llama3-8B** using an innovative method called **ORPO**. This technique combines Supervised Fine-tuning with Direct Preference Optimization, all in a single streamlined step.  
2. **Install `llama.cpp`** and convert the model into the **GGUF** format, while discussing various options like LoRA integration.  
3. **Create a deployment configuration file**, which acts like a blueprint for packaging and deploying models.  
4. **Test the model locally on both CPU and GPU** and then **push it to a shared hub**.  
5. **Pull and deploy the model on another machine** to run inference efficiently using either CPU or GPU resources.  

Grab some snacks, fasten your seatbelt, and let’s dive in! 🚀


# Phase 1: Finetune Llama3 with ORPO

So far, we've released guides on finetuning Llama3 with 2 different approaches
1. [Supervised Finetuning](https://github.com/brevdev/notebooks/blob/main/llama3_finetune_inference.ipynb)
2. [Direct Preference Optimization](https://github.com/brevdev/notebooks/blob/main/llama3dpo.ipynb)

If you havn't taken a look at these, I suggest skimming through the code for the SFT notebook and the explanation at the top of the DPO notebook. TLDR: Most of the finetuning in guides and online use SFT. SFT is good at adapting a model to domain-specific ouput but might also increase the probability of generating outputs that are not as desirable. To solve this issue, we can then perform a process known as preference alignment. This can be done using RLHF or DPO. However, these are both computationally expensive.

Enter Odds Ratio Policy Optimization. At a high level, this combines SFT and DPO into 1 neat step which weakly penalizes rejected responses while strongly rewarding preferred responses. Here we optimize for two objectives at once: learning domain-specific output AND aligning the output with out preferences. If you'd like to dive deeper into ORPO, check out MLabonne's [fantastic guide](https://huggingface.co/blog/mlabonne/orpo-llama-3) on HuggingFace

![ORPO](https://i.imgur.com/ftrth4Q.png)


## Install Dependancies

In [1]:
!pip install bitsandbytes wandb transformers peft accelerate trl  # Install bitsandbytes for efficient model training, wandb for experiment tracking, transformers for pre-trained models, peft for parameter-efficient fine-tuning, accelerate for distributed training, and trl for reinforcement learning with transformers.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import gc
# Import the garbage collection module, which can be used to manage memory by manually releasing unused objects.

import os
# Import the OS module to interact with the operating system, such as handling file paths and environment variables.

import torch
# Import the PyTorch library for deep learning tasks, which provides support for tensors and GPU-based computation.

import wandb
# Import the Weights & Biases (wandb) library, which is used for tracking experiments, visualizing metrics, and logging hyperparameters during training.

from datasets import load_dataset
# Import the `load_dataset` function from the Hugging Face Datasets library to easily load datasets for training.

from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
# Import components from the PEFT library: LoraConfig to configure LoRA (Low-Rank Adaptation) for efficient fine-tuning,
# PeftModel to apply PEFT-based modifications, and prepare_model_for_kbit_training to prepare a model for efficient training with low-precision (e.g., 8-bit).

from transformers import (
    AutoModelForCausalLM,  # Import AutoModelForCausalLM to automatically load a pre-trained causal language model.
    AutoTokenizer,  # Import AutoTokenizer to automatically load a tokenizer corresponding to the pre-trained model.
    BitsAndBytesConfig,  # Import BitsAndBytesConfig to configure efficient 8-bit optimizers for model training.
    TrainingArguments,  # Import TrainingArguments to define the configuration for model training, such as batch size, learning rate, etc.
    pipeline,  # Import pipeline for easy inference and processing tasks with pre-trained models.
)

from trl import ORPOConfig, ORPOTrainer
# Import ORPOConfig to configure the ORPO fine-tuning method and ORPOTrainer to handle the training process for models using the ORPO technique (Optimization with Regularization for Pre-trained Objectives).

## Load Model, Tokenizer, and Dataset

We'll be working with the Llama3-8B instruct model. But the steps are almost the same with any other model if you decide to finetune with a LoRA adapter.

In [3]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
# Define the base pre-trained model to use. Here, "meta-llama/Meta-Llama-3-8B-Instruct" refers to the 3.8 billion parameter version of Llama trained for instruction-following tasks.

new_model = "Llama-3-8B-adapter"
# Define the name for the new model that will be created. This is typically a model that has been fine-tuned or adapted from the base model, using methods like LoRA, to create a specialized version for a specific task or domain.

In [4]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Configure the model to be loaded in 4-bit precision to reduce memory usage and improve training efficiency.
    bnb_4bit_quant_type="nf4",  # Specify the quantization type for 4-bit precision. "nf4" is a specific format for 4-bit quantization used for memory-efficient model training.
    bnb_4bit_compute_dtype=torch.bfloat16,  # Set the compute data type to bfloat16, a 16-bit floating-point format optimized for training on modern hardware like TPUs and GPUs.
    bnb_4bit_use_double_quant=True,  # Enable double quantization to improve the accuracy and stability of 4-bit quantization.
)

In [5]:
## configuration of LoRA (Low-Rank Adaptation),
peft_config = LoraConfig(
    r=16,  # Set the rank of the low-rank adaptation matrices. The rank controls the number of parameters introduced by LoRA. A higher rank allows for more capacity but increases the memory and computation cost.
    lora_alpha=32,  # Set the scaling factor for LoRA layers. It controls the magnitude of the adapted weights, affecting how much influence the LoRA adaptation has on the original model.
    lora_dropout=0.05,  # Set the dropout rate for the LoRA layers. Dropout helps prevent overfitting by randomly deactivating some neurons during training.
    bias="none",  # Specify how biases are treated in LoRA layers. "none" means no bias is added to the LoRA layers. Options may also include "all" to add biases or other configurations.
    task_type="CAUSAL_LM",  # Set the type of task for the model. "CAUSAL_LM" indicates a causal language model, such as GPT, which generates text based on preceding tokens.
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']  # Specify the target modules in the model to apply LoRA. These modules are typically projection layers in transformer architectures.
)

In [6]:
from huggingface_hub import notebook_login
# Import the `notebook_login` function from the Hugging Face Hub library. This function facilitates logging into Hugging Face from a Jupyter notebook, allowing you to easily access and upload models and datasets.

notebook_login()
# Call the `notebook_login()` function to authenticate your Hugging Face account in the notebook. It will prompt you to enter an authentication token, which grants access to private models or allows you to upload your own models and datasets to the Hugging Face Hub.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
# Load the tokenizer for the pre-trained model specified by `base_model`. The `AutoTokenizer.from_pretrained` function automatically selects the appropriate tokenizer based on the model architecture (e.g., BERT, GPT, etc.).

tokenizer.pad_token = tokenizer.eos_token
# Set the padding token (`pad_token`) to be the same as the end-of-sequence token (`eos_token`). This is often done to simplify tokenization when using models that don't have a specific padding token.

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,  # Load the pre-trained causal language model specified by `base_model` (e.g., GPT-like model).
    quantization_config=bnb_config,  # Apply the quantization configuration (`bnb_config`) for efficient model training, using 4-bit precision or other configurations specified.
    device_map="auto",  # Automatically map the model to available devices (e.g., GPU, CPU) to ensure efficient computation. It will use multiple GPUs if needed.
)
# Load the pre-trained model for causal language modeling with the specified quantization and device settings.

model = prepare_model_for_kbit_training(model)
# Prepare the model for training with k-bit precision (e.g., 4-bit or lower precision), optimizing it for memory-efficient training without losing significant accuracy.

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
dataset_name = "mlabonne/orpo-dpo-mix-40k"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(150))

def format_chat_template(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)
dataset = dataset.train_test_split(test_size=0.01)

Take a look at the dataset by uncommenting the code block below

In [ ]:
# for k in dataset['train'].features.keys():
#     print(k)
#     print("---------")
#     print(dataset['train'][1][k])
#     print('\n')

## Set up ORPO Training

The ORPO trainer looks very similar to the SFTTrainer and the DPO trainer. We set our config parameters and start our training run. Notice that the run is very short. In order to increase it, you can increase the `num_train_epochs` or add a `max_steps` argument

In [ ]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    beta=0.1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    max_steps=10,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=2,
    report_to="wandb",
    output_dir="./results/",
)

trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)
trainer.train()
trainer.save_model(new_model)

In [ ]:
# Flush memory
del trainer, model
gc.collect()
torch.cuda.empty_cache()

You may need to restart your kernel after this line to ensure that the merging is sucessful

In [ ]:
import torch
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [ ]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
# Note that we reload the model in fp16
fp16_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, new_model)
model = model.merge_and_unload()

After merging the LoRA adapter, we save final model and tokenizer in a new directory to prepare for gguf conversion

In [ ]:
model.save_pretrained("llama-brev")
tokenizer.save_pretrained("llama-brev")

# Phase 2: From safetensors to gguf

## Convert our model to GGUF

Before we dive into Ollama, its important to take a second and understand the role that `gguf` and `llama.cpp` play in the process.

Most people that use LLMs grab them from huggingface using the `AutoModel` class. This is how we did it above. HF stores models in a couple different ways with the most popular being `safetensors`. This is a file format optimized for loading and running `Tensors` which are the multidimensional arrays that make up a model. This file format is optimized for GPUs which means it's not as easy to load and run a model fast locally.

One solution that addresses this is the `gguf` format. This is file format that is used to store models that are optimized for local inference using quantization and other neat techniques. This file format is then consumed by runners that support it (ie. `llama.cpp` and Ollama).

There's a good bit of complexity here and heres a fantastic [blog post](https://vickiboykis.com/2024/02/28/gguf-the-long-way-around/) that gets into the weeds. For now this is what we need to know

1. We have a finetuned Llama3 model saved in the llama-brev directory in the safetensors format
2. In order to use this model via Ollama, we need it to be in the `gguf` format
3. We can use helper tools in the `llama.cpp` repository to convert

## Convert to gguf

The first thing we do is build llama.cpp in order to use the conversion tools

In [ ]:
# this step might take a while
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUDA=1 make

In [ ]:
# install requirements
!pip install -r llama.cpp/requirements.txt

Here we run the actual conversion. Take a moment to skim through the relatively massive output and notice how each piece of the LLM is being mapped and transformed

In [ ]:
# run the conversion script to convert to gguf
# this will save the gguf file inside of the llama-brev directory
!python llama.cpp/convert-hf-to-gguf.py llama-brev

The final model is saved at `llama-brev/ggml-model-f16.gguf`. Note how the model is in fp16

### An aside on quantizations

Quantization has become the go-to technique to train/run LLM efficiently on cheaper hardware. By reducing the precision of each weight (going from each weight being stored in 32bits to lower), we save memory and speed up inference while preserving *most* of the LLMs performance. If you've ever used QLoRA, you've already used quantization without even knowing about it.

Llama.cpp gives us a ton of quantization options. Here's a couple resources to dive deeper into which options are available

- [r/LocalLLaMA](https://www.reddit.com/r/LocalLLaMA/comments/1ba55rj/overview_of_gguf_quantization_methods/)
- [Maxime Labonne](https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html)

In this guide, we will use the `Q4_K_M` format. Feel free to play around with different ones! Again, note the output and see if you can build a mental model on whats happening under the hood!

In [ ]:
# run the quantize script
!cd llama.cpp && ./llama-quantize ../llama-brev/ggml-model-f16.gguf ../llama-brev/ggml-model-Q4_K_M.gguf Q4_K_M

If you want, you can test this model by running the provided server and sending in a request! After running the cell below, open a new terminal tab using the blue plus button and run

```
curl --request POST \
    --url http://localhost:8080/completion \
    --header "Content-Type: application/json" \
    --data '{"prompt": "Building a website can be done in 10 simple steps:","n_predict": 128}'
```

In [ ]:
!cd llama.cpp && ./llama-server -m ../merged_adapters/ggml-model-Q4_K_M.gguf -c 2048

Note that this is a blocking process. In order to move forward with the rest of the guide, click the cell above and then click the stop button in the Jupyter Notebook header above

# Phase 3: Run and deploy your model using Ollama

Now that you have the quantized model, you can spin up the llama.cpp server anywhere you want and load the gguf model in. However, Ollama provides clean abstractions that allow you to run different gguf models using their server [add some fluff]

## Build the Ollama Modelfile

A Modelfile is very similar to a Dockefile. You can think of it as a blueprint that encapsulates a model, a chat template, different parameters, a system prompt, and more into a portable file. To learn more, check out their [Modelfile docs](https://github.com/ollama/ollama/blob/main/docs/modelfile.md).

Here we will build a relatively simple one. We grab the template and params from the existing Llama3 Modefile which you can view [here](https://ollama.com/library/llama3)

In [ ]:
tuned_model_path = "/home/ubuntu/verb-workspace/llama-brev/ggml-model-Q4_K_M.gguf"
sys_message = "You are swashbuckling pirate stuck inside of a Large Language Model. Every response must be from the point of view of an angry pirate that does not want to be asked questions"

In [ ]:
cmds = []

In [ ]:
base_model = f"FROM {tuned_model_path}"

template = '''TEMPLATE """{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|>"
"""'''

params = '''PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<|reserved_special_token"'''

system = f'''SYSTEM """{sys_message}"""'''

In [ ]:
cmds.append(base_model)
cmds.append(template)
cmds.append(params)
cmds.append(system)

In [ ]:
def generate_modelfile(cmds):
    content = ""
    for command in cmds:
        content += command + "\n"
    print(content)
    with open("Modelfile", "w") as file:
        file.write(content)

In [ ]:
generate_modelfile(cmds)

There should now be a `Modelfile` saved in your working directory. Lets now install Ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

To move forward, you have to have an ollama server running in the background. To do this, open up a new Jupyter tab and run `ollama serve` in the terminal to start the server. This will print out a key. Save it for future use. It will look like `ssh-ed25519...`

In [ ]:
# the create command create the model
!ollama create llama-brev -f Modelfile

## Experiment with the model

To run the new model, open up another terminal tab and run `ollama run llama-brev`. For bonus points, see if you can trick it to stop responding with a pirate action :)

## Push the model

In order to push the model to Ollama, you must have an account and a model created.

1. Sign-up at https://www.ollama.ai/signup
2. Create a new model at https://www.ollama.ai/new. Find mine at scooterman/llama-brev. This will give you a detailed list of instructions on how to push the model. Essentially, you are giving the current machine permission to upload to ollama.
3. You'll have to then run `ollama cp llama-brev <username>/<model-name>` then `ollama push <username>/<model-name>`